In [4]:
from langchain.retrievers import ContextualCompressionRetriever, CohereRagRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.chat_models import ChatCohere
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
import os


In [5]:

user_query = "Can the Agreement or any of its obligations be assigned?"
# Create cohere's chat model and embeddings objects
COHERE_API_KEY=os.environ.get("COHERE_API_KEY")
cohere_chat_model = ChatCohere(cohere_api_key=COHERE_API_KEY)
cohere_embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

In [6]:
raw_documents = TextLoader('./../prompts/context.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 3416, which is longer than the specified 1000
Created a chunk of size 4175, which is longer than the specified 1000
Created a chunk of size 4624, which is longer than the specified 1000


In [7]:
db = Chroma.from_documents(documents, cohere_embeddings)

In [8]:
cohere_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=db.as_retriever()
)
compressed_docs = compression_retriever.get_relevant_documents(user_query)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)

[Document(page_content="The Fees and the Workspace Expense for each month shall be payable by no later than ten (10) days from the beginning of the following month, against receipt by the Company of a duly issued tax invoice.\nIn addition, the Company shall reimburse Advisor for any reasonable and actual expenses incurred by the Advisor in connection with the performance of the Services, all in accordance with the Company’s policies as determined by the Board of Directors from time to time, and subject to Company's prior written consent for each specific expense, before Advisor incurs such expenses. As a condition to reimbursement, Advisor shall provide the Company with copies of all invoices, receipts and other evidence of expenditures as required by Company policy.\nAdvisor shall be solely responsible for any income taxes or other assessments made or imposed by any governmental authority on Advisor with respect to the Services rendered and the compensation received hereunder, and any

In [9]:
rag = CohereRagRetriever(llm=cohere_chat_model)

docs = rag.get_relevant_documents(
    user_query,
    source_documents=compressed_docs,
)

In [10]:
# Print the documents
for doc in docs[:-1]:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")

{'id': 'doc-0', 'snippet': "The Fees and the Workspace Expense for each month shall be payable by no later than ten (10) days from the beginning of the following month, against receipt by the Company of a duly issued tax invoice.\nIn addition, the Company shall reimburse Advisor for any reasonable and actual expenses incurred by the Advisor in connection with the performance of the Services, all in accordance with the Company’s policies as determined by the Board of Directors from time to time, and subject to Company's prior written consent for each specific expense, before Advisor incurs such expenses. As a condition to reimbursement, Advisor shall provide the Company with copies of all invoices, receipts and other evidence of expenditures as required by Company policy.\nAdvisor shall be solely responsible for any income taxes or other assessments made or imposed by any governmental authority on Advisor with respect to the Services rendered and the compensation received hereunder, and

In [11]:
# Print the final generation 
answer = docs[-1].page_content
print(answer)
# Print the final citations 
citations = docs[-1].metadata['citations']
print(citations)

The Agreement may not be assigned by Advisor for any reason whatsoever.
However, the Company may assign the Agreement to a successor of all or substantially all of its assets or business, provided the assignee has assumed the Company’s obligations under this Agreement.

Moreover, Advisor shall not appoint any sub-agents or delegates in connection with performance of the Services, nor assign any of his obligations or rights hereunder without obtaining the prior written consent of the Company.
[{'start': 45, 'end': 71, 'text': 'for any reason whatsoever.', 'document_ids': ['doc-0']}, {'start': 123, 'end': 186, 'text': 'successor of all or substantially all of its assets or business', 'document_ids': ['doc-0']}, {'start': 271, 'end': 496, 'text': 'Moreover, Advisor shall not appoint any sub-agents or delegates in connection with performance of the Services, nor assign any of his obligations or rights hereunder without obtaining the prior written consent of the Company.', 'document_ids': [

In [12]:
db.similarity_search(user_query,k=3)

[Document(page_content='Services:\nAdvisor shall provide to the Company, as an independent contractor, software development services, and / or any other services as agreed by the parties from time to time (the “Services”). Advisor shall not appoint any sub-agents or delegates in connection with performance of the Services, nor assign any of his obligations or rights hereunder without obtaining the prior written consent of the Company.\nWithout derogating from the foregoing, the Advisor shall not be required to provide the Services for a period of eighteen (18) days in each 12-month period.', metadata={'source': './../prompts/context.txt'}),
 Document(page_content="The Fees and the Workspace Expense for each month shall be payable by no later than ten (10) days from the beginning of the following month, against receipt by the Company of a duly issued tax invoice.\nIn addition, the Company shall reimburse Advisor for any reasonable and actual expenses incurred by the Advisor in connectio

In [13]:
from langchain.chat_models import ChatCohere
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm = ChatCohere(temperature=0.1, model='text-davinci-003')
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [14]:
qa.run(user_query)

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'I am not able to provide you with a legal advice, however, I can provide you with information that might be helpful. \n\nGenerally, in contract law, an agreement is considered personal between the parties involved and cannot be assigned to a third party without the consent of the other contracting parties. This is unless the contract specifically states so. This is done to preserve the rights and obligations of the original parties to the contract. However, there are some exceptions to this rule, for example in the case of mergers or acquisitions, where a third party may be assigned an agreement by operation of law. \n\nIt is recommended that you review the terms of the specific agreement you are dealing with and consult a legal professional specialized in contracts for precise advice. \n\nThis is meant to provide you with some helpful introductory insights into the general aspects of contract assignment.'

In [15]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            "You are an advanced Contract Q&A assistant designed specifically for Lizzy AI's Contract Advisor project." 
"Your primary role is to facilitate queries and provide informative responses regarding contract-related inquiries."
"Use the following pieces of retrieved context to answer the question. "
"If you don't know the answer, just say that you don't know."

        )
    )
]

In [16]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [17]:
response = agent(user_query)

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "assignment provision"
}
```
Observation: In the context of law, an assignment provision refers to a clause within a contract that outlines the process and conditions for the transfer of rights or obligations from one party to another. This clause specifies who can assign the contract, any restrictions on such assignments, and whether consent is required from other parties for the transfer to take place.

Here are some standard components of an assignment provision: 

1. Assignment Clause: This clause explicitly states that the contract can be assigned, prohibiting any inference against the permissibility of assignment.

2. Who Can Assign: It specifies which parties to the contract can assign their rights and obligations. This could include all parties or only certain individuals or entities.

3. Procedure for Assignment: This outlines the steps required for a valid assignment, such as

In [18]:
response

{'input': 'Can the Agreement or any of its obligations be assigned?',
 'chat_history': [],
 'output': 'assignment clause prohibits inferring against the permissibility of assignment and specifies which parties can assign their rights and obligations. Other standard components include the procedure for assignment, binding effect, consent requirements, and notice provisions. In the absence of a contract, legislation may provide default rules regarding the assignability of contracts.'}